### Group Assignment 3 (20%)


#### Group Members

-   Ezlan Zulfiqree bin Hashim (17192056)
-   Nurulnadiah binti Sukerman (S2178467)
-   Nur Dhania Kamalia Kamarol (S2121964)


#### Import the necessary libraries


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn import preprocessing
import matplotlib.pyplot as plt

#### Define the necessary variables


In [ ]:
batch_size = 64  # Specify the batch size and create data loaders
input_size = 16  # The number of features in your dataset
hidden1_size = 10  # Number of units in the first hidden layer
hidden2_size = 5  # Number of units in the second hidden layer
output_size = (
    2  # Output size for softmax (assuming binary classification, change if needed)
)
learning_rate = 0.025  # Stochastic Gradient Descent optimizer learning rate
num_epochs = 100  # You can adjust the number of epochs as needed

#### Instantiate fetus training and testing datasets


In [ ]:
# Set random seed for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)


# Define a custom dataset class for the fetus dataset
class FetusDataset(Dataset):
    def __init__(self, csv_file):
        le = preprocessing.LabelEncoder()
        csv_data = pd.read_csv(csv_file)
        csv_features = csv_data.iloc[:, 1:].values
        csv_labels = csv_data.iloc[:, 0].values
        csv_labels = le.fit_transform(csv_labels)
        self.features = torch.tensor(csv_features)
        self.labels = torch.tensor(csv_labels)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


# Define the paths to the training and testing CSV files
# train_csv_file = "Fetus_trainingdata.csv"
# test_csv_file = "Fetus_testingdata.csv"

# Use the scaled version of the training and testing CSV files for the modified MLP model
train_csv_file = "Normalize_train.csv"
test_csv_file = "Normalize_test.csv"

# Create training and testing datasets
train_dataset = FetusDataset(train_csv_file)
test_dataset = FetusDataset(test_csv_file)


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# You can now use train_loader and test_loader to iterate through your datasets in batches.

#### Instantiate MLP class with init and forward methods


##### Default Setting: Activation function (HL1: ReLU / HL2: Softmax)


In [ ]:
# Original


class MLP(nn.Module):
    def __init__(self, input_size, hidden1_size, hidden2_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

In [ ]:
# Default


# class MLP(nn.Module):
#     def __init__(self, input_size, hidden1_size, hidden2_size, output_size):
#         super(MLP, self).__init__()
#         self.fc1 = nn.Linear(input_size, hidden1_size)
#         self.relu_i_h1 = nn.ReLU()
#         self.fc2 = nn.Linear(hidden1_size, hidden2_size)
#         self.softmax_h1_h2 = nn.Softmax(dim=1)
#         self.fc3 = nn.Linear(hidden2_size, output_size)

#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.relu_i_h1(x)
#         x = self.fc2(x)
#         x = self.softmax_h1_h2(x)
#         x = self.fc3(x)
#         return x

##### One Hidden Layer Only. Activation function (HL1: ReLU)


In [ ]:
# One Hidden Layer Only. Activation function (HL1: ReLU)


# class MLP(nn.Module):
#     def __init__(self, input_size, hidden1_size, hidden2_size, output_size):
#         super(MLP, self).__init__()
#         self.fc1 = nn.Linear(input_size, hidden1_size)
#         self.relu_i_h1 = nn.ReLU()
#         self.fc3 = nn.Linear(hidden1_size, output_size)

#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.relu_i_h1(x)
#         x = self.fc3(x)
#         return x

##### Activation function (HL1: ReLU / HL2: ReLU)


In [ ]:
# Activation function (HL1: ReLU / HL2: ReLU)


# class MLP(nn.Module):
#     def __init__(self, input_size, hidden1_size, hidden2_size, output_size):
#         super(MLP, self).__init__()
#         self.fc1 = nn.Linear(input_size, hidden1_size)
#         self.relu_i_h1 = nn.ReLU()
#         self.fc2 = nn.Linear(hidden1_size, hidden2_size)
#         self.relu_h1_h2 = nn.ReLU()
#         self.fc3 = nn.Linear(hidden2_size, output_size)

#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.relu_i_h1(x)
#         x = self.fc2(x)
#         x = self.relu_h1_h2(x)
#         x = self.fc3(x)
#         return x

##### Activation function (HL1: Softmax / HL2: Softmax)


In [ ]:
# Activation function (HL1: Softmax / HL2: Softmax)


# class MLP(nn.Module):
#     def __init__(self, input_size, hidden1_size, hidden2_size, output_size):
#         super(MLP, self).__init__()
#         self.fc1 = nn.Linear(input_size, hidden1_size)
#         self.Softmax_i_h1 = nn.Softmax(dim=1)
#         self.fc2 = nn.Linear(hidden1_size, hidden2_size)
#         self.softmax_h1_h2 = nn.Softmax(dim=1)
#         self.fc3 = nn.Linear(hidden2_size, output_size)

#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.Softmax_i_h1(x)
#         x = self.fc2(x)
#         x = self.softmax_h1_h2(x)
#         x = self.fc3(x)
#         return x

##### Activation function (HL1: Softmax / HL2: ReLU)


In [ ]:
# Activation function (HL1: Softmax / HL2: ReLU)


# class MLP(nn.Module):
#     def __init__(self, input_size, hidden1_size, hidden2_size, output_size):
#         super(MLP, self).__init__()
#         self.fc1 = nn.Linear(input_size, hidden1_size)
#         self.Softmax_i_h1 = nn.Softmax(dim=1)
#         self.fc2 = nn.Linear(hidden1_size, hidden2_size)
#         self.relu_h1_h2 = nn.ReLU()
#         self.fc3 = nn.Linear(hidden2_size, output_size)

#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.Softmax_i_h1(x)
#         x = self.fc2(x)
#         x = self.relu_h1_h2(x)
#         x = self.fc3(x)
#         return x

##### Activation function (HL1: ReLU)


In [ ]:
# Activation function (HL1: ReLU)


# class MLP(nn.Module):
#     def __init__(self, input_size, hidden1_size, hidden2_size, output_size):
#         super(MLP, self).__init__()
#         self.fc1 = nn.Linear(input_size, hidden1_size)
#         self.relu_i_h1 = nn.ReLU()
#         self.fc3 = nn.Linear(hidden1_size, output_size)

#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.relu_i_h1(x)
#         x = self.fc3(x)
#         return x

##### Activation function (HL1: Softmax)


In [ ]:
#  Activation function (HL1: Softmax)


# class MLP(nn.Module):
#     def __init__(self, input_size, hidden1_size, hidden2_size, output_size):
#         super(MLP, self).__init__()
#         self.fc1 = nn.Linear(input_size, hidden1_size)
#         self.softmax_i_h1 = nn.Softmax(dim=1)
#         self.fc3 = nn.Linear(hidden1_size, output_size)

#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.softmax_i_h1(x)
#         x = self.fc3(x)
#         return x

#### Train MLP model with fetus training data


In [ ]:
# Instantiate the MLP model
model = MLP(input_size, hidden1_size, hidden2_size, output_size)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = optim.SGD(
    model.parameters(), lr=learning_rate
)  # Stochastic Gradient Descent optimizer

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch [{epoch + 1}/{num_epochs}] Loss: {total_loss / len(train_loader)}")

print("Training completed.")

#### Evaluate MLP model performance with fetus testing data


In [ ]:
# Evaluate the model on the testing data
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on the testing data: {accuracy:.2f}%")

#### Plot training and testing loss, training and testing accuracy across epochs


In [ ]:
# Lists to store training and testing loss across epochs
train_losses = []
test_losses = []

# Lists to store training and testing accuracy across epochs
train_accuracies = []
test_accuracies = []

# Training loop with recording loss and accuracy
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # Computing accuracy during training
        _, predicted_train = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted_train == labels).sum().item()

    # Calculate training accuracy and append to list
    train_accuracy = 100 * correct_train / total_train
    train_accuracies.append(train_accuracy)

    # Calculate average training loss and append to list
    train_losses.append(total_loss / len(train_loader))

    # Evaluate the model on the testing data
    model.eval()
    correct_test = 0
    total_test = 0

    with torch.no_grad():
        test_loss = 0.0
        for inputs, labels in test_loader:
            outputs = model(inputs.float())
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            # Computing accuracy during testing
            _, predicted_test = torch.max(outputs, 1)
            total_test += labels.size(0)
            correct_test += (predicted_test == labels).sum().item()

        # Calculate testing accuracy and append to list
        test_accuracy = 100 * correct_test / total_test
        test_accuracies.append(test_accuracy)

        # Calculate average testing loss and append to list
        test_losses.append(test_loss / len(test_loader))


# Plotting training and testing loss across epochs
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label="Training Loss")
plt.plot(test_losses, label="Testing Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training and Testing Loss")
plt.legend()
plt.show()

# Plotting training and testing accuracy across epochs
plt.figure(figsize=(10, 5))
plt.plot(train_accuracies, label="Training Accuracy")
plt.plot(test_accuracies, label="Testing Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Training and Testing Accuracy")
plt.legend()
plt.show()